In [1]:
from Evaluation import Matrix
from ModelFactory import model_insightface
import numpy as np
import tqdm
import torch
import torch.nn as nn

from dataloader.dataset import CALFWDataset
from torch.utils.data import DataLoader
from torchvision import transforms as T

In [2]:
import importlib
importlib.reload(model_insightface)

<module 'ModelFactory.model_insightface' from 'c:\\Users\\Ben\\Documents\\cv_final\\CV-Final-FaceRecognition\\BaselineScript\\ModelFactory\\model_insightface.py'>

In [10]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
import os
import csv


class FileDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.id2fn = []
        self.fn2id = {}

        files = os.listdir(root)

        for f in files:
            self.fn2id[f] = len(self.id2fn)
            self.id2fn.append(f)

    def __getitem__(self, item):
        image = Image.open(os.path.join(self.root, self.id2fn[item]))

        if self.transform is not None:
            image = self.transform(image)
        
        return image

    def __len__(self):
        return len(self.id2fn)

In [25]:
model = model_insightface.MobileFaceNet(512).to('cuda').eval()
model.load_state_dict(torch.load('./Model/model_mobilefacenet.pth'))

<All keys matched successfully>

In [43]:
model = model_insightface.GroupMobileFaceNet(512).to('cuda').eval()
model.load_state_dict(torch.load('ckpt/model-best_gp_11.pth'))

<All keys matched successfully>

# Similarity

In [44]:
normalization = T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
dataset = FileDataset(
    '../data/calfw/FGNet_500/',
    transform=T.Compose([T.Resize(112), T.ToTensor(), normalization]))
dataloader = DataLoader(dataset, 64)

In [45]:
with torch.no_grad():
    features = []
    for i, pics in enumerate(tqdm.tqdm(dataloader, file=sys.stdout, desc='Evaluating: ', leave=False)):
        pics = pics.to('cuda')
        features.append(model(pics))
    features = torch.cat(features, dim=0).cpu()

pd_pair = []

for i in range(len(dataset)):
    pd_pair.append(Matrix.get_cosine_similarity(features[i:i + 1], features[i + 1:])[0])

pd_table = torch.zeros((len(dataset), len(dataset)))

for i, vec in enumerate(pd_pair):
    pd_table[i, i+1:] = vec
    pd_table[i+1:, i] = vec

In [46]:
file_pairs = []
fn2id = dataset.fn2id
with open('../data/calfw/ForTesting/Test_PairList.csv',
                'r', encoding='utf-8', newline='') as f:
    for fn1, fn2 in csv.reader(f):
        file_pairs.append((fn2id[fn1], fn2id[fn2]))

In [47]:
with open('Test_team08_results.csv',
                'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    for (id1, id2) in file_pairs:
        writer.writerow([pd_table[id1][id2].item()])

# Grouping

In [48]:
normalization = T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
dataset = FileDataset(
    '../data/calfw/ForGrouping/',
    transform=T.Compose([T.Resize(112), T.ToTensor(), normalization]))
dataloader = DataLoader(dataset, 64)

In [49]:
with torch.no_grad():
    features = []
    for i, pics in enumerate(tqdm.tqdm(dataloader, file=sys.stdout, desc='Evaluating: ', leave=False)):
        pics = pics.to('cuda')
        features.append(model(pics))
    features = torch.cat(features, dim=0).cpu()

pd_pair = []

for i in range(len(dataset)):
    pd_pair.append(Matrix.get_cosine_similarity(features[i:i + 1], features[i + 1:])[0])

pd_table = torch.zeros((len(dataset), len(dataset)))

for i, vec in enumerate(pd_pair):
    pd_table[i, i+1:] = vec
    pd_table[i+1:, i] = vec

In [50]:
group = torch.zeros((len(dataset)))
used = torch.zeros((len(dataset)), dtype=torch.bool)

for i in range(20):
    dist = None
    for j in range(len(used)):
        if used[j]:
            continue
        target_id = j
        dist = pd_table[i, :]
        used[j] = True
        group[j] = i
        break
    if dist == None:
        print("error")
    dist[used] = -2
    for _ in range(1, 6):
        target_id = torch.argmax(dist)
        dist = torch.max(torch.stack([pd_table[i, :], dist], dim=0), dim=0)[0]
        used[target_id] = True
        dist[used] = -2
        group[target_id] = i

In [51]:
torch.unique(group, return_counts=True)

(tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19.]),
 tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]))

In [52]:
file_order = []
fn2id = dataset.fn2id
with open('../data/calfw/ForTesting/Test_BonusList.csv',
                'r', encoding='utf-8', newline='') as f:
    for [fn] in csv.reader(f):
        file_order.append(fn2id[fn])

In [53]:
with open('Test_team08_bonus_results.csv',
                'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    for id in file_order:
        writer.writerow([group[id].item()])